In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(),
                    nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[-0.1762, -0.2958,  0.4635, -0.1417, -0.0232, -0.2091,  0.0495,  0.1768,
          0.4733, -0.0209],
        [-0.0598, -0.2054,  0.3616, -0.1320, -0.0821, -0.1292,  0.2441,  0.1986,
          0.1914,  0.0525]], grad_fn=<AddmmBackward0>)

### custom block

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256) # hidden layer
        self.out = nn.Linear(256, 10) # output layer
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[-0.1753,  0.0377,  0.1089, -0.1401, -0.1446,  0.0554,  0.1079, -0.0037,
          0.0439, -0.1397],
        [-0.1707,  0.1966,  0.0572, -0.0609, -0.0223, -0.0872,  0.1073,  0.0959,
         -0.0320, -0.0265]], grad_fn=<AddmmBackward0>)

### sequential block

In [7]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [8]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0522,  0.4178, -0.4670,  0.0819,  0.0879,  0.0936, -0.0607,  0.0942,
         -0.0388,  0.0028],
        [-0.0566,  0.1904, -0.3250,  0.1622,  0.0557,  0.1610, -0.0620,  0.1118,
         -0.0869, -0.0337]], grad_fn=<AddmmBackward0>)

### execute code in forward propagation

In [11]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [12]:
net = FixedHiddenMLP()
net(X)

tensor(-0.1195, grad_fn=<SumBackward0>)

In [13]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20))
chimera(X)

tensor([[-0.2309, -0.1204, -0.1771,  0.1974, -0.1977, -0.0431, -0.1161, -0.1930,
         -0.1524, -0.1783,  0.0652,  0.1452, -0.1017,  0.1811,  0.1234, -0.2629,
          0.2320,  0.1486, -0.0298, -0.0974],
        [-0.2344, -0.1320, -0.1836,  0.1977, -0.1615, -0.0486, -0.1270, -0.2181,
         -0.1423, -0.1713,  0.0371,  0.1407, -0.0592,  0.1311,  0.1679, -0.1979,
          0.2048,  0.1284,  0.0117, -0.1031]], grad_fn=<AddmmBackward0>)